In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#NOTE: IS_SUCCESSFUL should be my target

#FEATURES will be: AFFILIATION, ORGANIZATION, USE_CASE, STATUS, INCOME_AMT, ASK_AMT,

#Dropped will be: EIN, NAME, APPLICATION_TYPE, SPECIAL, CONSIDERATIONS, CLASSIFICATION

charity_relevent_df = charity_df.drop(columns=["EIN","NAME","APPLICATION_TYPE","SPECIAL_CONSIDERATIONS","CLASSIFICATION"])
charity_relevent_df.head()

,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,ProductDev,Association,1,0,5000,1
1,Independent,Preservation,Co-operative,1,1-9999,108590,1
2,CompanySponsored,ProductDev,Association,1,0,5000,0
3,CompanySponsored,Preservation,Trust,1,10000-24999,6692,1
4,Independent,Heathcare,Trust,1,100000-499999,142590,1


In [3]:
#Lets make sure that none of the categorical data needs bucketing


# Generate our categorical variable list
charity_cat = charity_relevent_df.dtypes[charity_relevent_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_relevent_df[charity_cat].nunique()

AFFILIATION     6
USE_CASE        5
ORGANIZATION    4
INCOME_AMT      9
dtype: int64

In [4]:
# Check the unique value counts to see if bucketing required
charity_relevent_df.USE_CASE.value_counts()


Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [5]:
charity_relevent_df.ORGANIZATION.value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [6]:
# I AM GOING TO DECIDE NOT TO BUCKET FOR SIMPLICITY

In [7]:
#Lets encode data now:

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_relevent_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_relevent_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
charity_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'AFFILIATION_CompanySponsored',
       'AFFILIATION_Family/Parent', 'AFFILIATION_Independent',
       'AFFILIATION_National', 'AFFILIATION_Other', 'AFFILIATION_Regional',
       'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M'],
      dtype='object')

In [10]:
# Remove the target features from the data
y = charity_relevent_df['IS_SUCCESSFUL'].values
X = charity_df.drop(columns=["IS_SUCCESSFUL"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# WE WILL TRY FIRST TO USE A DEEP LEARNING MODEL.
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [12]:
#train and evaluate the DLL

# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 3s 105us/sample - loss: 0.6237 - accuracy: 0.6857
Epoch 2/50
25724/25724 [==============================] - 2s 76us/sample - loss: 0.6031 - accuracy: 0.6964
Epoch 3/50
25724/25724 [==============================] - 2s 63us/sample - loss: 0.6012 - accuracy: 0.6964
Epoch 4/50
25724/25724 [==============================] - 2s 63us/sample - loss: 0.6002 - accuracy: 0.6974
Epoch 5/50
25724/25724 [==============================] - 1s 56us/sample - loss: 0.5997 - accuracy: 0.6985
Epoch 6/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5992 - accuracy: 0.6986
Epoch 7/50
25724/25724 [==============================] - 1s 56us/sample - loss: 0.5990 - accuracy: 0.6986
Epoch 8/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5986 - accuracy: 0.6986
Epoch 9/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5984 - accuracy: 0.6994
Epoch 10/50
2

In [14]:
#Okay so this model topped out at about 0.7000 at one point. There is still alot of loss.

In [15]:
#We will try again, but try a different activation function: THIS time we'll try a sigmoid activation function in the HIDDEN LAYERS

# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 65us/sample - loss: 0.6461 - accuracy: 0.6383
Epoch 2/50
25724/25724 [==============================] - 1s 45us/sample - loss: 0.6100 - accuracy: 0.6942
Epoch 3/50
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6069 - accuracy: 0.6985
Epoch 4/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6048 - accuracy: 0.6988
Epoch 5/50
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6031 - accuracy: 0.6991
Epoch 6/50
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6019 - accuracy: 0.6987
Epoch 7/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6007 - accuracy: 0.6992
Epoch 8/50
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5999 - accuracy: 0.6998
Epoch 9/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5991 - accuracy: 0.6993
Epoch 10/50
25

In [17]:
#Okay, this sucked too! Lets try going back to a "Relu" activation function in the Hidden Layers, but we'll attempt to add more inputs

In [18]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 70us/sample - loss: 0.6217 - accuracy: 0.6830
Epoch 2/50
25724/25724 [==============================] - 1s 49us/sample - loss: 0.6023 - accuracy: 0.6972
Epoch 3/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.6004 - accuracy: 0.6983
Epoch 4/50
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5998 - accuracy: 0.6964
Epoch 5/50
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5992 - accuracy: 0.6973
Epoch 6/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5992 - accuracy: 0.6983
Epoch 7/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5986 - accuracy: 0.6993
Epoch 8/50
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5987 - accuracy: 0.6989
Epoch 9/50
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5983 - accuracy: 0.6983
Epoch 10/50
25

In [20]:
#Still having a ton of loss. The fitting is still not great in that it's still at 0.65

#Lets try increasing the epochs next

In [21]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=300) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

00
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5952 - accuracy: 0.7023
Epoch 121/300
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5950 - accuracy: 0.7023
Epoch 122/300
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5951 - accuracy: 0.7023
Epoch 123/300
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5952 - accuracy: 0.7024
Epoch 124/300
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5952 - accuracy: 0.7022
Epoch 125/300
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5951 - accuracy: 0.7023
Epoch 126/300
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5952 - accuracy: 0.7026
Epoch 127/300
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5952 - accuracy: 0.7028
Epoch 128/300
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5952 - accuracy: 0.7025
Epoch 129/300
25724/2

In [22]:
# Increasing the epochs still did not work very well for our model

In [ ]:
#Lets try one more time: this time we will keep the enlarged Hidden Layers but we will scale back the epochs. We will now CHANGE the output activation function to ALSO be "Relu"

In [25]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 90us/sample - loss: 0.6139 - accuracy: 0.6865
Epoch 2/50
25724/25724 [==============================] - 1s 54us/sample - loss: 0.6013 - accuracy: 0.6980
Epoch 3/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.6003 - accuracy: 0.6987
Epoch 4/50
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5996 - accuracy: 0.6995
Epoch 5/50
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5989 - accuracy: 0.6995
Epoch 6/50
25724/25724 [==============================] - 1s 56us/sample - loss: 0.5986 - accuracy: 0.6994
Epoch 7/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5983 - accuracy: 0.7011
Epoch 8/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5983 - accuracy: 0.7004
Epoch 9/50
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5981 - accuracy: 0.7002
Epoch 10/50
25

In [27]:
# The results STILL can't cross past 70%

In [28]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 3s 99us/sample - loss: 1.6098 - accuracy: 0.6118
Epoch 2/50
25724/25724 [==============================] - 2s 60us/sample - loss: 0.6990 - accuracy: 0.6933
Epoch 3/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.6810 - accuracy: 0.6944
Epoch 4/50
25724/25724 [==============================] - 2s 58us/sample - loss: 0.6779 - accuracy: 0.6955
Epoch 5/50
25724/25724 [==============================] - 2s 60us/sample - loss: 0.6727 - accuracy: 0.6969
Epoch 6/50
25724/25724 [==============================] - 2s 62us/sample - loss: 0.6711 - accuracy: 0.6964
Epoch 7/50
25724/25724 [==============================] - 2s 61us/sample - loss: 0.6715 - accuracy: 0.6969
Epoch 8/50
25724/25724 [==============================] - 2s 59us/sample - loss: 0.6742 - accuracy: 0.6959
Epoch 9/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.6731 - accuracy: 0.6967
Epoch 10/50
25

In [30]:
# Nope!